## Additional information about the PAO1 and TCGA pathway-pathway networks

In [1]:
import itertools
import os

import numpy as np
import pandas as pd
from scipy.stats import fisher_exact
from statsmodels.sandbox.stats.multicomp import multipletests

In [2]:
data_dir = os.path.join("..", "data")
pao1_data = os.path.join(data_dir, "pao1_data")
tcga_data = os.path.join(data_dir, "tcga_data")

In [3]:
pao1_kegg_edges_file = os.path.join(
    pao1_data, "eADAGE_analysis", "permutation_test_n=10000", "filtered_network.tsv")
tcga_pid_edges_file = os.path.join(
    tcga_data, "NMF_analysis", "permutation_test_n=10000", "filtered_network.tsv")

kegg_pathways = os.path.join(pao1_data, "pseudomonas_KEGG_terms.txt")
pid_pathways = os.path.join(tcga_data, "PID_pathway_definitions.txt")

In [4]:
def load_pathway_definitions_file(pathways_file, shorten_pathway_name):
    pathway_definitions_df = pd.read_table(
        pathways_file, header=None, names=["pw", "size", "genes"])
    pathway_definitions_df["genes"] = pathway_definitions_df["genes"].map(
        lambda x: x.split(";"))
    pathway_definitions_df.set_index("pw", inplace=True)

    pathway_definitions_map = {}
    for index, row in pathway_definitions_df.iterrows():
        pathway = shorten_pathway_name(index)
        pathway_definitions_map[pathway] = set(row["genes"])
    return pathway_definitions_map

In [5]:
pao1_kegg_network = pd.read_table(pao1_kegg_edges_file)
pao1_kegg_network.head()

,pw0,pw1,weight
0,Ribosome PAO1,"Ribosome, bacteria",26.089640
1,Phosphonate and phosphinate metabolism PAO1,Phosphonate transport system,25.455394
2,"Macrolide resistance, MacAB-TolC transporter",Zinc transport system,23.170151
3,Phosphonate and phosphinate metabolism PAO1,Type II general secretion pathway,21.168310
4,Glycine betaine/proline transport system,"Glycine, serine and threonine metabolism PAO1",19.221697


In [6]:
# shorten the PAO1 KEGG pathway names so that they match the ones
# we use in the publication
def shorten_pao1_kegg(pathway_name):
    REMOVE_SUFFIX = "- Pseudomonas aeruginosa PAO1"
    pathway_short = None
    split_label = pathway_name.split(" ", 1)
    if len(split_label) > 1:
        pathway_short = split_label[1]
    else:
        pathway_short = split_label[0]
    if REMOVE_SUFFIX in pathway_short:
        remove_from_index = pathway_short.index(REMOVE_SUFFIX)
        return "{0}PAO1".format(pathway_short[:remove_from_index])
    return pathway_short.strip()

kegg_pathways_dict = load_pathway_definitions_file(
    kegg_pathways, shorten_pao1_kegg)

## PAO1 KEGG network summary statistics

In [7]:
pathways_in_kegg_network = set(
    pao1_kegg_network["pw0"].tolist() + pao1_kegg_network["pw1"].tolist())

n_overlapping = 0
edge_jaccards = []
for index, row in pao1_kegg_network.iterrows():
    pw0_genes = kegg_pathways_dict[row["pw0"]] 
    pw1_genes = kegg_pathways_dict[row["pw1"]]
    overlap = pw0_genes & pw1_genes
    if overlap:
        n_overlapping += 1
    jaccard = float(len(overlap)) / (len(pw0_genes) + len(pw1_genes) - len(overlap))
    edge_jaccards.append(jaccard)        
        
print("For the PAO1 KEGG pathway-pathway co-occurrence network:")
print("Number of distinct pathways in the network: {0}".format(
    len(pathways_in_kegg_network)))
print("Number of edges in the network: {0}".format(
    pao1_kegg_network.shape[0]))
print("Number of edges in the network where the 2 pathways "
      "share genes: {0} (average similarity by Jaccard Index: {1:.4})".format(
          n_overlapping, np.average(edge_jaccards)))

For the PAO1 KEGG pathway-pathway co-occurrence network:
Number of distinct pathways in the network: 89
Number of edges in the network: 203
Number of edges in the network where the 2 pathways share genes: 35 (average similarity by Jaccard Index: 0.03482)


### Comparison to a network based only on overlapping genes

In [8]:
def create_overlap_based_network(pathway_genes_map, alpha=0.05):
    all_genes = set().union(*pathway_genes_map.values())
    n_genes = len(all_genes)
    
    pvalues_list = []
    pathway_pairs = list(itertools.combinations(
        pathway_genes_map.keys(), 2))
    for pw0, pw1 in pathway_pairs:
        pw0_genes = pathway_genes_map[pw0]
        pw1_genes = pathway_genes_map[pw1]
        n_common = len(pw0_genes & pw1_genes)
        n_pw0 = len(pw0_genes - pw1_genes)
        n_pw1 = len(pw1_genes - pw0_genes)
        n_all_other_genes = n_genes - n_pw0 - n_pw1 - n_common
        contingency_table = np.array(
            [[n_common, n_pw0],
             [n_pw1, n_all_other_genes]])
        _, pvalue = fisher_exact(
            contingency_table, alternative="greater")
        pvalues_list.append(pvalue)
    
    below_alpha, qvalues, _, _ = multipletests(
        pvalues_list, alpha=alpha, method="fdr_bh")
    
    significant_edges = [(pathway_pairs[ix], qvalues[ix]) for ix, is_below 
                         in enumerate(below_alpha) if is_below]
    return significant_edges

In [9]:
significant_edges = create_overlap_based_network(kegg_pathways_dict, alpha=0.05)
print("Number of edges in the PAO1 KEGG overlap-based network: {0}\n".format(
    len(significant_edges)))
sorted_significant_edges = sorted(significant_edges, key=lambda tup: tup[1])

show_N_edges = 150
print("Pathway\t\tAdjusted p-value")
for edge, adjusted_pval in sorted_significant_edges[:show_N_edges]:
    print("{0}:\t{1:.4f}".format(edge, adjusted_pval))

Number of edges in the PAO1 KEGG overlap-based network: 406

Pathway		Adjusted p-value
('Ribosome PAO1', 'Ribosome, bacteria'):	0.0000
('Fatty acid metabolism PAO1', 'Fatty acid biosynthesis PAO1'):	0.0000
('Citrate cycle (TCA cycle) PAO1', 'Citrate cycle (TCA cycle, Krebs cycle)'):	0.0000
('Bacterial secretion system PAO1', 'Type II general secretion pathway'):	0.0000
('Citrate cycle, second carbon oxidation, 2-oxoglutarate => oxaloacetate', 'Citrate cycle (TCA cycle, Krebs cycle)'):	0.0000
('Valine, leucine and isoleucine degradation PAO1', 'Propanoate metabolism PAO1'):	0.0000
('Biotin metabolism PAO1', 'Pimeloyl-ACP biosynthesis, BioC-BioH pathway, malonyl-ACP => pimeloyl-ACP'):	0.0000
('Fatty acid biosynthesis PAO1', 'Fatty acid biosynthesis, elongation'):	0.0000
('Fatty acid degradation PAO1', 'Fatty acid metabolism PAO1'):	0.0000
('Citrate cycle (TCA cycle) PAO1', 'Citrate cycle, second carbon oxidation, 2-oxoglutarate => oxaloacetate'):	0.0000
('Fatty acid degradation PAO1', 'V

## TCGA PID network summary statistics

In [10]:
tcga_pid_network = pd.read_table(tcga_pid_edges_file)
tcga_pid_network.head()

,pw0,pw1,weight,features
0,E2F,PLK1,6.543478,19.0 29.0 39.0 44.0 59.0 114.0 115.0 158.0
1,AURORA B,E2F,6.531936,19.0 29.0 39.0 44.0 59.0 114.0 115.0 158.0
2,AURORA B,PLK1,6.341347,19.0 29.0 39.0 44.0 59.0 114.0 115.0 158.0
3,E2F,FOXM1,5.974458,19.0 29.0 39.0 44.0 59.0 115.0 158.0
4,AURORA B,FOXM1,5.873026,19.0 29.0 39.0 44.0 59.0 115.0 158.0


In [11]:
# shorten the PID pathway names so that they match the ones
# we use in the publication
def shorten_tcga_pid(pathway_name):
    REMOVE_SUFFIX = "PATHWAY"
    pathway_short = None
    split_on_underscores = pathway_name.split("_")
    if split_on_underscores[-1] == REMOVE_SUFFIX:
        pathway_short = " ".join(split_on_underscores[:-1])
    else:
        pathway_short = " ".join(split_on_underscores)
    return pathway_short

pid_pathways_dict = load_pathway_definitions_file(
    pid_pathways, shorten_tcga_pid)

In [12]:
pathways_in_pid_network = set(
    tcga_pid_network["pw0"].tolist() + tcga_pid_network["pw1"].tolist())

n_overlapping = 0
edge_jaccards = []
for index, row in tcga_pid_network.iterrows():
    pw0_genes = pid_pathways_dict[row["pw0"]] 
    pw1_genes = pid_pathways_dict[row["pw1"]]
    overlap = pw0_genes & pw1_genes
    if overlap:
        n_overlapping += 1
    jaccard = float(len(overlap)) / (len(pw0_genes) + len(pw1_genes) - len(overlap))
    edge_jaccards.append(jaccard)

print("For the TCGA PID pathway-pathway co-occurrence network:")
print("Number of distinct pathways in the network: {0}".format(
    len(pathways_in_kegg_network)))
print("Number of edges in the network: {0}".format(
    pao1_kegg_network.shape[0]))
print("Number of edges in the network where the 2 pathways "
      "share genes: {0} (average similarity by Jaccard Index: {1:.4})".format(
          n_overlapping, np.average(edge_jaccards)))

For the TCGA PID pathway-pathway co-occurrence network:
Number of distinct pathways in the network: 89
Number of edges in the network: 203
Number of edges in the network where the 2 pathways share genes: 96 (average similarity by Jaccard Index: 0.05783)


### Comparison to a network based only on overlapping genes

In [13]:
significant_edges = create_overlap_based_network(pid_pathways_dict, alpha=0.05)
print("Number of edges in the PID overlap-based network: {0}\n".format(
    len(significant_edges)))
sorted_significant_edges = sorted(significant_edges, key=lambda tup: tup[1])

show_N_edges = 150
print("Pathway\t\tAdjusted p-value")
for edge, adjusted_pval in sorted_significant_edges[:show_N_edges]:
    print("{0}:\t{1:.4f}".format(edge, adjusted_pval))

Number of edges in the PID overlap-based network: 3826

Pathway		Adjusted p-value
('TCR', 'CD8 TCR'):	0.0000
('ERBB1 DOWNSTREAM', 'PDGFRB'):	0.0000
('IL8 CXCR2', 'IL8 CXCR1'):	0.0000
('AVB3 INTEGRIN', 'SYNDECAN 1'):	0.0000
('ENDOTHELIN', 'LYSOPHOSPHOLIPID'):	0.0000
('CDC42', 'RAC1'):	0.0000
('GMCSF', 'IL2 1PATHWAY'):	0.0000
('FCER1', 'BCR 5PATHWAY'):	0.0000
('GMCSF', 'IL3'):	0.0000
('CDC42', 'ERBB1 DOWNSTREAM'):	0.0000
('INTEGRIN1', 'INTEGRIN3'):	0.0000
('S1P S1P3', 'S1P S1P2'):	0.0000
('NFAT TFPATHWAY', 'TCR CALCIUM'):	0.0000
('ERBB1 DOWNSTREAM', 'ERBB2 ERBB3'):	0.0000
('ERBB1 RECEPTOR PROXIMAL', 'PDGFRB'):	0.0000
('SHP2', 'IL2 1PATHWAY'):	0.0000
('KIT', 'EPO'):	0.0000
('ER NONGENOMIC', 'THROMBIN PAR1'):	0.0000
('S1P S1P3', 'S1P S1P1'):	0.0000
('TXA2PATHWAY', 'IL8 CXCR1'):	0.0000
('IL2 PI3K', 'IL2 STAT5'):	0.0000
('TRAIL', 'FAS'):	0.0000
('LYSOPHOSPHOLIPID', 'ER NONGENOMIC'):	0.0000
('ENDOTHELIN', 'S1P S1P2'):	0.0000
('IL12 2PATHWAY', 'IL12 STAT4'):	0.0000
('MET', 'FAK'):	0.0000
('THR